In [1]:
import tensorflow as tf

In [2]:
from __future__ import absolute_import,  division , print_function
tf.enable_eager_execution()

In [3]:
tf.executing_eagerly()

True

In [4]:
x = [[2.]]

In [5]:
m = tf.matmul(x,x)

In [6]:
print('hello,{}'.format(m))

hello,[[4.]]


In [7]:
a = tf.constant([[1,2],[3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [8]:
#broadcasting 
b = tf.add(a,1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [9]:
print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [10]:
import numpy as np
c = np.multiply(a,b)
print(c)
print(a.numpy())

[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


In [11]:
tfe = tf.contrib.eager

In [87]:
def fizzbuz(max_num):
    counter = tf.constant(0)
    for num in range(max_num):
        num = tf.constant(num)
        if int(num%3)==0 and int(num % 5)==0:
            print('FizzBuzz')
        elif int(num % 3 ) == 0:
            print('Fizz')
        else:
            print(num)
        counter  += 1
    return counter 


In [88]:
class MySimpleLayer(tf.keras.layers.Layer):
    def __init__(self, output_units):
        self.output_units = output_units

    def build(self, input):
        
    # The build method gets called the first time your layer is used.
    # Creating variables on build() allows you to make their shape depend
    # on the input shape and hence remove the need for the user to specify
    # full shapes. It is possible to create variables during __init__() if
    # you already know their full shapes.
        self.kernel = self.add_variable(          
            "kernel", [input.shape[-1], self.output_units])

    def call(self, input):
    # Override call() instead of __call__ so we can perform some bookkeeping.
        return tf.matmul(input, self.kernel)


In [89]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
])


In [90]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=10)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, input):
        """Run the model."""
        result = self.dense1(input)
        result = self.dense2(result)
        result = self.dense2(result)  # reuse variables from dense2 layer
        return result

model = MNISTModel()


In [91]:
w  = tfe.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
    
grad = tape.gradient(loss,[w])
print(grad)

[<tf.Tensor: id=152482, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


In [35]:
# a toy dataset of points around 3 * x + 2 
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise 

def prediction(input, weight, bias):
    return input * weight + bias 

# A loss function using mean squared error
def loss(weights,biases):
    error = prediction(training_inputs,weights,biases) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value  = loss(weights, biases)
    return tape.gradient(loss_value,[weights,biases])

train_steps  =  200
learning_rate = 0.01
# start with arbitary values for W and B on the same batch of data
W = tfe.Variable(5.)
B = tfe.Variable(10.)


print("Initial loss {:.3f}".format(loss(W,B)))

for i in range(train_steps):
    dW, dB = grad(W,B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dW * learning_rate)
    if i % 20 == 0 : 
        print("Loss at step {:03d} : {:.3f}".format(i,loss(W,B)))
        
print("Final loss: {:.3f}".format(loss(W,B)))
print("W ={}, B= {}".format(W.numpy(),B.numpy()))



Initial loss 69.559
Loss at step 000 : 68.683
Loss at step 020 : 55.583
Loss at step 040 : 48.066
Loss at step 060 : 43.599
Loss at step 080 : 40.867
Loss at step 100 : 39.160
Loss at step 120 : 38.077
Loss at step 140 : 37.381
Loss at step 160 : 36.932
Loss at step 180 : 36.639
Final loss: 36.456
W =2.953416347503662, B= 7.9534196853637695


In [73]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')
    def predict(self, inputs):
        return inputs * self.W + self.B

# a toy dataset of points around 3 * x + 2 
NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise 

# The loss function to be optimized 
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model,inputs,targets)
    return tape.gradient(loss_value, [model.W, model.B])

#Define : 
# 1.  A model
# 2. Derivatives of a loss function with respect to model parameters 
# 3. A strategy for updating the variables based on  the derivatives 
writer = tf.contrib.summary.create_file_writer('logdir/')
global_step=tf.train.get_or_create_global_step() 



model = Model() 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("Initial loss : {:.3f}".format(loss(model,training_inputs, training_outputs)))

# Training loop

for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                            global_step=tf.train.get_or_create_global_step())

    if i % 20 == 0:
        print ("Loss at step {:03d}: {:.3f}".format(i,loss(model, training_inputs,training_outputs)))
        
        
print("Final loss: {:.3f}".format(loss(model,training_inputs,training_outputs)))
print("W ={}, B={}".format(model.W.numpy(), model.B.numpy()))
                

Initial loss : 70.271
Loss at step 000: 67.490
Loss at step 020: 30.318
Loss at step 040: 13.952
Loss at step 060: 6.741
Loss at step 080: 3.561
Loss at step 100: 2.158
Loss at step 120: 1.539
Loss at step 140: 1.265
Loss at step 160: 1.144
Loss at step 180: 1.091
Loss at step 200: 1.067
Loss at step 220: 1.057
Loss at step 240: 1.052
Loss at step 260: 1.050
Loss at step 280: 1.049
Final loss: 1.049
W =2.9589593410491943, B=2.011157989501953


In [60]:
# tfe.Checkpoint can save and restore tfe.Variables to and from checkpoints:

x = tfe.Variable(10.)
checkpoint = tfe.Checkpoint(x=x) # save a 'x'

x.assign(2.) # assign a new value to the variables  and save.
save_path = checkpoint.save('ckpt/')

x.assign(11.) # change the variable after saving

# restore values from the checkpoint
checkpoint.restore(save_path)

print(x) # --> 2.0

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


In [74]:
import os
model = Model()
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
checkpoint_dir = "ckpt/"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tfe.Checkpoint(optimizer=optimizer,
                      model=model,
                      optimizer_step=tf.train.get_or_create_global_step())

root.save(file_prefix=checkpoint_prefix)
# or
root.restore(tf.train.latest_checkpoint(checkpoint_dir))


In [75]:
#tfe.metrics are stored as objects. 
#Update a metric by passing the new data to the callable,
#and retrieve the result using the tfe.metrics.result method

m = tfe.metrics.Mean('loss')
m(0)
m(5)
m.result() 

<tf.Tensor: id=151581, shape=(), dtype=float64, numpy=2.5>

In [76]:
m([8,9])

<tf.Tensor: id=151589, shape=(), dtype=int32, numpy=17>

In [77]:
@tf.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.log(1 + e), grad

grad_log1pexp = tfe.gradients_function(log1pexp)

# As before, the gradient computation works fine at x = 0.
grad_log1pexp(0.)  # => [0.5]

# And the gradient computation also works at x = 100.
grad_log1pexp(100.)  # => [1.0]


[<tf.Tensor: id=151611, shape=(), dtype=float32, numpy=1.0>]

In [79]:
import time

def measure(x, steps):
  # TensorFlow initializes a GPU the first time it's used, exclude from timing.
  tf.matmul(x, x)
  start = time.time()
  for i in range(steps):
    x = tf.matmul(x, x)
    _ = x.numpy()  # Make sure to execute op and not just enqueue it
  end = time.time()
  return end - start

shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

# Run on CPU:
with tf.device("/cpu:0"):
  print("CPU: {} secs".format(measure(tf.random_normal(shape), steps)))

# Run on GPU, if available:
if tfe.num_gpus() > 0:
  with tf.device("/gpu:0"):
    print("GPU: {} secs".format(measure(tf.random_normal(shape), steps)))
else:
  print("GPU: not found")


Time to multiply a (1000, 1000) matrix by itself 200 times:
CPU: 6.226000547409058 secs
GPU: not found


In [86]:

def my_py_func(x):
    x = tf.matmul(x, x)  # You can use tf ops
    print(x)  # but it's eager!
    return x

with tf.Session() as sess:    
    x = tf.placeholder(dtype=tf.float32)
    # Call eager function in graph!
    pf = tfe.py_func(my_py_func, [x], tf.float32)
    sess.run(pf, feed_dict={x: [[2.0]]})  # [[4.0]]


ValueError: tf.enable_eager_execution must be called at program startup.